In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Liver_cirrhosis"
cohort = "GSE182065"

# Input paths
in_trait_dir = "../../input/GEO/Liver_cirrhosis"
in_cohort_dir = "../../input/GEO/Liver_cirrhosis/GSE182065"

# Output paths
out_data_file = "../../output/preprocess/Liver_cirrhosis/GSE182065.csv"
out_gene_data_file = "../../output/preprocess/Liver_cirrhosis/gene_data/GSE182065.csv"
out_clinical_data_file = "../../output/preprocess/Liver_cirrhosis/clinical_data/GSE182065.csv"
json_path = "../../output/preprocess/Liver_cirrhosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression profiling of prognostic liver signature in clinical fibrotic liver tissues cultured with various anti-fibrotic and chemopreventive agents"
!Series_summary	"Background/Aims: There is a major unmet need to assess prognostic impact of anti-fibrotics in clinical trials due to the slow rate of liver fibrosis progression. We aimed to develop a surrogate biomarker to predict future fibrosis progression."
!Series_summary	"Methods: A Fibrosis Progression Signature (FPS) was defined to predict fibrosis progression within 5 years in HCV and NAFLD patients with no to minimal fibrosis at baseline (n=421), and validated in an independent NAFLD cohort (n=78). The FPS was used to assess response to 13 candidate anti-fibrotics in organotypic ex vivo cultures of clinical fibrotic liver tissues (n=78), and cenicriviroc in NASH patients enrolled in a clinical trial (n=19, NCT02217475). A serum-protein-based surrogate FPS (FPSec) was developed and technical

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information and series title, this dataset appears to contain gene expression data
# The summary mentions a 20-gene Fibrosis Progression Signature (FPS) and gene expression profiling
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Liver cirrhosis):
# The background information indicates this is a study on liver fibrosis, but there's no explicit cirrhosis indicator
# in the sample characteristics. However, it's a study about fibrotic liver tissues, so all samples have some level
# of fibrosis but we can't distinguish cirrhosis specifically.
trait_row = None  # Cannot determine cirrhosis status from the available data

# For age:
# No age information is provided in the sample characteristics
age_row = None

# For gender:
# No gender information is provided in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
# Since trait_row is None, we don't need to define convert_trait, but we'll create a placeholder function
def convert_trait(value):
    # This function won't be used since trait_row is None
    return None

def convert_age(value):
    # This function won't be used since age_row is None
    return None

def convert_gender(value):
    # This function won't be used since gender_row is None
    return None

# 3. Save Metadata
# The trait_row is None, meaning trait data is not available
is_trait_available = False if trait_row is None else True

# Initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we should skip this substep


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Liver_cirrhosis/GSE182065/GSE182065_series_matrix.txt.gz
Gene data shape: (192, 293)
First 20 gene/probe identifiers:
Index(['AARS', 'ABLIM1', 'ACOT2', 'ACSM3', 'ACTR2', 'ADD3', 'ADH5', 'ADH6',
       'ADRA2B', 'AEBP1', 'AKAP13', 'AKR1A1', 'AKR1D1', 'ALAS1', 'ALDH9A1',
       'ANKRD46', 'ANXA1', 'ANXA3', 'AOX1', 'AP1B1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the data
# Looking at the first 20 gene identifiers: 'AARS', 'ABLIM1', 'ACOT2', etc.
# These appear to be standard human gene symbols
# No mapping to gene symbols is required as they're already in that format

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Check if trait_row is None (indicating no clinical data is available)
if trait_row is None:
    print("No clinical data available for this dataset, skipping clinical data processing.")
    
    # Validate and save cohort information with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use this data without clinical features
        df=pd.DataFrame(),  # Empty DataFrame since we have no linked data
        note="Dataset contains gene expression data from cell lines with HCV infection, which is not appropriate for liver cirrhosis trait analysis."
    )
    
    print("Dataset is not usable for liver cirrhosis analysis due to lack of clinical data. No linked data file saved.")
else:
    # If clinical data is available, proceed with the linking and processing
    # 2. Link the clinical and genetic data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty after missing value handling")

    # 5. Validate and save cohort information
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note="Dataset contains gene expression data for liver fibrosis progression, which is relevant to liver cirrhosis research."
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")

Gene data shape before normalization: (192, 293)
Gene data shape after normalization: (191, 293)
Normalized gene expression data saved to ../../output/preprocess/Liver_cirrhosis/gene_data/GSE182065.csv
No clinical data available for this dataset, skipping clinical data processing.
Abnormality detected in the cohort: GSE182065. Preprocessing failed.
Dataset is not usable for liver cirrhosis analysis due to lack of clinical data. No linked data file saved.
